Haal gebruikers op die films hebben gerate. Er ontstaat een RDD[(userId, movieId, rating)].

In [101]:
userdata = sc.textFile("../../../schooldata/nieuw/ratings_train.csv")

firstline = userdata.first()
userdata = userdata.filter(lambda x: x != firstline)

data_normalized = userdata.map(lambda x: x.split("::"))
user_mov_rating = data_normalized.map(lambda x: (int(x[0]), int(x[1]), float(x[2]))).cache()

user_mov_rating.take(5)

[(11973, 11, 3.0),
 (11973, 14, 4.0),
 (11973, 32, 2.0),
 (11973, 36, 3.0),
 (11973, 110, 5.0)]

Haal films op uit de database en map ze in een RDD zodat er een RDD[(movieId, title)] ontstaat.

In [4]:
moviedata = sc.textFile("../../../schooldata/nieuw/movies.csv")

firstline = moviedata.first()
movies_normalized = moviedata.filter(lambda x: x != firstline)\
.map(lambda x: x.split(','))

mov_id_name = movies_normalized.map(lambda x: (x[0], x[1])).cache()

mov_id_name.take(5)

[('1', 'Toy Story (1995)'),
 ('2', 'Jumanji (1995)'),
 ('3', 'Grumpier Old Men (1995)'),
 ('4', 'Waiting to Exhale (1995)'),
 ('5', 'Father of the Bride Part II (1995)')]

Initialiseer de benodigde ALS parameters en laat ALS trainen. Vervolgens een voorspelde lijst in predictions stoppen, die we laten joinen met de test data split.

In [120]:
from pyspark.mllib.recommendation import ALS
import math

# verdeel rdd data 60% cls, 20% validatie en 20% tests
cls, validation, test = every_rating.randomSplit([6, 2, 2], seed=None)

# laat ALS op het cls rdd trainen.
trained_prototype = ALS.train(cls, rank = 8, seed = 5, iterations = 10, lambda_ = 0.01)

test_predictions = test.map(lambda x: (x[0], x[1]))

# de rating wordt voorspeld voor userId en movieId door het getrainde ALS prototype 
# en word gemapt als ((userId, movieId), rating).
predictions = trained_prototype.predictAll(test_predictions)\
.map(lambda x: ((x[0], x[1]), x[2]))

# de test rdd joint de predictions rdd en dan krijgen we ((userId, movieId), (rating, rating)).
rnp = test.map(lambda x: ((int(x[0]), int(x[1])), float(x[2])))\
.join(predictions)

computable_rnp = rnp.map(lambda x: (x[1][0] - x[1][1])**2)

# bereken de Root Mean Square Error bij de rnp data.
error = math.sqrt(computable_rnp.mean())

print('Met rank %s is de RMSE: %s' % (rank, error))

Met rank 10 is de RMSE: 0.8202497107829302


In [119]:
predictions.filter(lambda x: x[1] > 4).takeOrdered(20, key=lambda x: x[0][0])

AttributeError: 'list' object has no attribute 'filter'

Werk het movie aanbevelingsalgoritme uit, zodat users die bepaalde movies nog niet gezien hebben, maar er wel in geinteresseerd kunnen zijn die films ook aanbevolen krijgen. Het gaat hierbij om movies met rating >= 4.

In [122]:
movieId_rating = every_rating.map(lambda x: (x[1], x[2])).groupByKey()

movieId = mov_id_name.map(lambda x: x[0])

# map (movieId, (len(ratings), avg_rating)).
# avg_rating krijgen we door alle bijbehorende ratings op te sommen en 
# die te delen door de totale hoeveelheid (count) bijbehorende ratings
# en dat casten we naar een float.
# *met bijbehorende bedoelen we die bij de movieId horen
movieId_avgRating = movieId_rating\
.map(lambda x: (x[0], (len(x[1]), float(sum(i for i in x[1])) / len(x[1]))))

n = mov_id_name.collect()
mi = movieId.collect()

user_no_rating = sc.parallelize([m for m in n if m not in m])

predictions = predictions.collect()

recommendations = sorted(predictions, key=lambda x: x[1], reverse=True)[:50]

for i in range(len(recommendations)):
    print("%2d: %s" % (i + 1, movies[recommendations[i][1]]))

AttributeError: 'list' object has no attribute 'collect'